In [0]:
%run "/Workspace/Users/ruchika.b.mhetre@v4c.ai/vstone_project/vstone_databricks_pipeline/src/notebooks/00_Setup/project_config"

# Table Initialization

In [0]:
%sql
-- Create the initial Silver table if it doesn't exist
CREATE TABLE IF NOT EXISTS vstone_project.db_project.silver_car_sales
AS 
SELECT *, current_timestamp() AS load_dt, 'initial_load' AS source
FROM vstone_project.db_project.bronze_transactions; -- Replace with your actual bronze table

# Simulating Incremental Data (Temp View)

In [0]:
%sql
SELECT count(*) AS silver_row_count FROM vstone_project.db_project.silver_car_sales;

# The MERGE INTO (Upsert Logic)

In [0]:
%sql
-- 1. SET CONTEXT
USE CATALOG vstone_project;
USE SCHEMA db_project;

-- 2. CREATE THE INCREMENTAL SOURCE (CHUNK 2)
-- This creates the session-scoped view immediately before use
CREATE OR REPLACE TEMP VIEW incremental_updates_view AS
SELECT 
    id, 
    price * 1.15 AS price, -- Simulate a 15% price increase for late-arriving data
    mileage,
    currency,
    transaction_date,
    current_timestamp() AS load_dt,
    'chunk_2_drop' AS source
FROM silver_car_sales
LIMIT 20;

-- 3. EXECUTE THE MERGE INTO (UPSERT LOGIC)
-- This handles late-arriving data by updating matching IDs [Day 7 Requirement]
MERGE INTO silver_car_sales AS target
USING incremental_updates_view AS source
ON target.id = source.id
WHEN MATCHED THEN
  UPDATE SET 
    target.price = source.price,
    target.load_dt = source.load_dt,
    target.source = source.source
WHEN NOT MATCHED THEN
  INSERT (id, price, mileage, currency, transaction_date, load_dt, source)
  VALUES (source.id, source.price, source.mileage, source.currency, source.transaction_date, source.load_dt, source.source);

-- 4. VERIFY THE RESULTS
SELECT source, count(*) as record_count 
FROM silver_car_sales 
GROUP BY source;